In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import re
import shutil
from sklearn.model_selection import train_test_split

import tensorflow as tf
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.layers.experimental import preprocessing

In [ ]:
load_model = tf.keras.models.load_model("model/vegecare_model_mobilenetv2finetune.h5")

In [ ]:
classes = [
    "Cabai__Healthy",
    "Cabai__Leaf_Curl",
    "Cabai__Leaf_Spot",
    "Cabai__Whitefly",
    "Cabai__Yellowish",
    "Kembang Kol__Bacterial_Spot_Rot",
    "Kembang Kol__Black_Rot",
    "Kembang Kol__Downy_Mildew",
    "Kembang Kol__Healthy",
    "Lettuce_Bacterial",
    "Lettuce__Fungal",
    "Lettuce__Healthy",
    "Sawi_Hama_Ulat_Grayak",
    "Sawi_Healthy",
    "Terong__Healthy_Leaf",
    "Terong__Insect_Pest_Disease",
    "Terong__Leaf_Spot_Disease",
    "Terong__Mosaic_Virus_Disease",
    "Terong__Small_Leaf_Disease",
    "Terong__White_Mold_Disease",
    "Terong__Wilt_Disease",
    "Timun__Anthracnose",
    "Timun__Bacterial_Wilt",
    "Timun__Belly_Rot",
    "Timun__Downy_Mildew",
    "Timun__Fresh_Leaf",
    "Timun__Gummy_Stem_Blight",
    "Timun__Pythium_Fruit_Rot",
    "Tomato__Bacterial_Spot",
    "Tomato__Early_Blight",
    "Tomato__Healthy",
    "Tomato__Late_Blight",
    "Tomato__Leaf_Mold",
    "Tomato__Septoria_Leaf_Spot",
    "Tomato__Spider_Mites Two-Spotted_Spider_Mite",
    "Tomato__Target_Spot",
    "Tomato__Tomato_Mosaic_Virus",
    "Tomato__Tomato_Yellow_Leaf_Curl_Virus",
]

In [ ]:
# Paths
train_dir = "/content/train"
val_dir = "/content/val"
test_dir = "/content/test"

In [ ]:
def create_datasets(path_dataset, image_size=(224, 224), batch_size=20):
    AUTOTUNE = tf.data.AUTOTUNE
    normalization_layer = tf.keras.layers.Rescaling(1.0 / 255)

    train_ds = (
        tf.keras.utils.image_dataset_from_directory(
            train_dir,
            seed=0,
            batch_size=batch_size,
            label_mode="categorical",
            image_size=image_size,
        )
        .map(lambda x, y: (normalization_layer(x), y))
        .prefetch(buffer_size=AUTOTUNE)
    )

    validation_ds = (
        tf.keras.utils.image_dataset_from_directory(
            val_dir,
            seed=0,
            batch_size=batch_size,
            label_mode="categorical",
            image_size=image_size,
        )
        .map(lambda x, y: (normalization_layer(x), y))
        .prefetch(buffer_size=AUTOTUNE)
    )

    test_ds = (
        tf.keras.utils.image_dataset_from_directory(
            test_dir,
            seed=0,
            batch_size=batch_size,
            label_mode="categorical",
            image_size=image_size,
        )
        .map(lambda x, y: (normalization_layer(x), y))
        .prefetch(buffer_size=AUTOTUNE)
    )

    return train_ds, validation_ds, test_ds


train_ds, validation_ds, test_ds = create_datasets("/content/")

In [ ]:
img_path = "/content/test/Tomato__Tomato_Yellow_Leaf_Curl_Virus/0a3f5fbb-1b3b-4b3b-8b3b-3b3b3b3b3b3b___YLCV_NREC 2908.JPG"

In [ ]:
from tensorflow.keras.preprocessing import image

img = image.load_img(img_path, target_size=(224, 224))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)

In [ ]:
img

In [ ]:
predictions = load_model.predict(img_array)

In [ ]:
print("Predicted Class Index:", CLASSES[predicted_class_index])

In [ ]:
test_images = []
test_labels = []

for img, label in test_ds.unbatch().as_numpy_iterator():
    img = img.astype("uint8")
    test_images.append(img)
    test_labels.append(label)

test_images = np.array(test_images)
test_labels = np.array(test_labels)

# Evaluate the model on the validation dataset
loss, acc = load_model.evaluate(test_images, test_labels)

print(f"Validation Loss: {loss}, Validation Accuracy: {acc}")